<a href="https://colab.research.google.com/github/alirezakhaleghianaghizi/MLC/blob/main/AI%D9%80Course%D9%80Challenge%D9%80NLP%D9%80Phase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl">
<font face="B Nazanin" size=5>
    <b>بسم الله الرحمن الرحیم</b>
</font>
</div>

<div dir="rtl">
<font face="B Nazanin" size=5>
    <b>چالش درس هوش مصنوعی - پردازش زبان طبیعی</b>
    <br>
    <b>فاز اول</b>
</font>
</div>

<div dir="rtl">
<font face="B Nazanin" size=4>
نوت بوکی که پیش روی شماست، مراحل انجام فاز اول چالش درس هوش مصنوعی در قسمت پردازش زبان طبیعی را تعیین کرده است. در این فاز، شما با استفاده مجموعه‌ای از محتوای متن صفحات وب به همراه برچسب موضوع هر یک، مدلی را توسعه می‌دهید که توانایی دسته‌بندی متن ورودی را به یکی از 16 برچسب مجموعه‌ داده داشته باشد.
<br>
<br>
<b>چند نکته مهم</b>
<ul >
<font face="B Nazanin" size=4>
<li>
می‌توانید از کتابخانه‌های با پیاده‌سازی آماده مدل‌های پایه (مانند transformers) استفاده کنید.
</li>
<li>
توجه کنید که تمام کتابخانه های مورد نیاز را نصب کنید و از نصب شدن آن ها اطمینان حاصل کرده و نام آن ها در فایل requirements ذکر کنید.
</li>
<li>
در صورت نیاز به نصب یک کتابخانه یا دسترسی به یک فایل، مراحل نصب و دانلود آن (از یک محل عمومی یا فایل ارسال شده به همراه نوت‌بوک) می‌بایست در نوت‌بوک وجود داشته باشد.
</li>
<li>
تمامی فایل‌های مرتبط به پروژه که دارای حجم کمی هستند، باید به همراه نوت‌بوک در قالب یک فایل زیپ ارسال شوند. فایل‌هایی که حجم زیادی دارند (مانند فایل ذخیره شده یک مدل که بیشتر از ۲۰۰ مگابایت باشد) را در یک محل عمومی مانند گوگل درایو آپلود کرده و لینک دسترسی به آن را در نوت بوک قرار دهید.
</li>
</font>
</ul>
</font>
</div>

<div dir="rtl">
<font face="B Nazanin" size=4>
<b>بارگذاری و پیش‌پردازش داده</b>
<br>
در ابتدا می‌بایست داده‌ها را از فایل با نام dataset_phase1 بارگذاری کنید. سپس برای اینکه بتوانید مدل مناسبی را آموزش دهید،  داده‌ها را پیش پردازش کنید. این قسمت می‌تواند شامل موارد متنوعی باشد؛ از جمله حذف عبارت‌های اضافی موجود در انتهای متن‌ها، حذف یا جداسازی علائم نگارشی، توکن‌بندی کلمات، جایگزینی مواردی مثل اعداد، تاریخ‌ها و آدرس‌‌های الکترونیکی با توکن‌های خاص، حذف فاصله‌های اضافه و ... . برای این منظور، می‌توانید از کتاب‌خانه‌هایی مانند hazm هم استفاده کنید.
سپس برای اینکه بتوانید ارزیابی مناسبی از مدل خود داشته باشید، داده‌ها را به دو قسمت آموزش  (train) و ارزیابی  (validation) تقسیم کنید.
</font>
</div>

In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import drive
import string
import re
from nltk.stem.porter import PorterStemmer
import nltk
!pip install hazm
from hazm import *

# data must be uploded to the drive
drive.mount('/content/drive')
filename='/content/drive/MyDrive/ml/phase_1_train.csv'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df = pd.read_csv(filename)
df.head()

,text,class_id,class_name
0,تداوم فاز انتظاری در بازار خودرو/دنا پلاس امرو...,21,خودرو
1,موتورسیکلت آلمانی با پیشرانه وی 6 | فیلم - نیو...,21,خودرو
2,کدام خودروها در مهر 1401 امکان واردات دارند؟ -...,21,خودرو
3,\r\n خودرو از 0 تا 12762 میلیون تومان\r...,21,خودرو
4,آبروریزی جنجالی سایپا در نمایشگاه خودرو گزارش ...,21,خودرو


In [3]:
X = df.drop('class_id', axis=1)
X = df.drop('class_name', axis=1)
y_id = df['class_id']
y_name = df['class_name']


In [4]:
string.punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
X['clean_msg']= X['text'].apply(lambda x:remove_punctuation(x))

def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result
X['number_removed']=X['clean_msg'].apply(lambda x: remove_numbers(x))

def tokenization(text):
    tokens = word_tokenize(text)
    return tokens
#applying function to the column
X['msg_tokenied']= X['number_removed'].apply(lambda x: tokenization(x))
#importing the Stemming function from nltk library

lemmatizer = Lemmatizer()

#defining a function for lemmatizer
def mylemmatizer(text):

    lemma_text = [lemmatizer.lemmatize(word) for word in text]
    return lemma_text
X['msg_lemmatized']=X['msg_tokenied'].apply(lambda x: mylemmatizer(x))
X.head()


,text,class_id,clean_msg,number_removed,msg_tokenied,msg_lemmatized
0,تداوم فاز انتظاری در بازار خودرو/دنا پلاس امرو...,21,تداوم فاز انتظاری در بازار خودرودنا پلاس امروز...,تداوم فاز انتظاری در بازار خودرودنا پلاس امروز...,"[تداوم, فاز, انتظاری, در, بازار, خودرودنا, پلا...","[تداوم, فاز, انتظار, در, بازار, خودرودنا, پلاس..."
1,موتورسیکلت آلمانی با پیشرانه وی 6 | فیلم - نیو...,21,موتورسیکلت آلمانی با پیشرانه وی 6 فیلم نیوزی...,موتورسیکلت آلمانی با پیشرانه وی فیلم نیوزین...,"[موتورسیکلت, آلمانی, با, پیشرانه, وی, فیلم, نی...","[موتورسیکلت, آل, با, پیشرانه, وی, فیلم, نیوزین..."
2,کدام خودروها در مهر 1401 امکان واردات دارند؟ -...,21,کدام خودروها در مهر 1401 امکان واردات دارند؟ ...,کدام خودروها در مهر امکان واردات دارند؟ مجله...,"[کدام, خودروها, در, مهر, امکان, واردات, دارند,...","[کدام, خودرو, در, مهر, امکان, واردات, داشت#دار..."
3,\r\n خودرو از 0 تا 12762 میلیون تومان\r...,21,\r\n خودرو از 0 تا 12762 میلیون تومان\r...,\r\n خودرو از تا میلیون تومان\r\n ...,"[\r, خودرو, از, تا, میلیون, تومان\r, نمایشگاه,...","[\r, خودرو, از, تا, میلیون, تومان\r, نمایشگاه,..."
4,آبروریزی جنجالی سایپا در نمایشگاه خودرو گزارش ...,21,آبروریزی جنجالی سایپا در نمایشگاه خودرو گزارش ...,آبروریزی جنجالی سایپا در نمایشگاه خودرو گزارش ...,"[آبروریزی, جنجالی, سایپا, در, نمایشگاه, خودرو,...","[آبروریزی, جنجال, سایپا, در, نمایشگاه, خودرو, ..."


below is not completed yet

we shoud count each word frequency of happened in each group and by those number we can model the nlp for persian text

below is code for two group of text in english for tweets

In [ ]:
def create_freq(tweets):
  freq = {}
  for tweet in tweets:
    for word in process_tweet(tweet):
      if word in freq:
        freq[word] += 1
      else:
        freq[word] = 1
  return freq

#positive tweets and  negetiv tweets are two group of our text
freq_pos = create_freq(positive_tweets)
freq_neg = create_freq(negative_tweets)


def convert_tweet_to_vector(tweets, freq_pos, freq_neg):
  vectors = np.zeros((len(tweets), 2))
  for index, tweet in enumerate(tweets):
    #process tweet return the tokenized of sentence for our data we should use X["msg_lemmatized"]
    for word in process_tweet(tweet):
      vectors[index][0] += freq_pos.get(word, 0)
      vectors[index][1] += freq_neg.get(word, 0)
    vectors[index] /= np.linalg.norm(vectors[index])
  return vectors

pos_tweet_vectors = convert_tweet_to_vector(positive_tweets, freq_pos, freq_neg)
neg_tweet_vectors = convert_tweet_to_vector(negative_tweets, freq_pos, freq_neg)

print(pos_tweet_vectors[0:5])
print(neg_tweet_vectors[0:5])



X_train = np.concatenate((pos_tweet_vectors[0:4500] , neg_tweet_vectors[0:4500]), axis=0)
Y_train = np.array([1] * 4500 + [0] * 4500)

X_test = np.concatenate((pos_tweet_vectors[4500:] , neg_tweet_vectors[4500:]), axis=0)
Y_test = np.array([1] * 500 + [0] * 500)

print(f'X_train shape={X_train.shape}, Y_train shape={Y_train.shape}')
print(f'X_test shape={X_test.shape}, Y_test shape={Y_test.shape}')

0       [تداوم, فاز, انتظار, در, بازار, خودرودنا, پلاس...
1       [موتورسیکلت, آل, با, پیشرانه, وی, 6, فیلم, نیو...
2       [کدام, خودرو, در, مهر, 1401, امکان, واردات, دا...
3       [\r, خودرو, از, 0, تا, 12762, میلیون, تومان\r,...
4       [آبروریزی, جنجال, سایپا, در, نمایشگاه, خودرو, ...
                              ...                        
2697    [۵۷, چرا, لاک‌پشت‌ها, لاک, بزرگ, و, سنگین, داش...
2698    [راه, ابریشم, فیزیک, آلاءبا, تدریس, آقا, طلوع,...
2699    [استفاده, از, مخمر, نان, برای, تولید, یک, آنتی...
2700    [بزرگ, مهاجرت, روی, زمین, هر, روز, زیر, تاریکی...
2701    [تیمون, چه, حیوانی, #است, ؟, ویکی, دانا, ورود,...
Name: msg_lemmatized, Length: 2702, dtype: object


In [ ]:
TEST_SIZE = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y_id, test_size=TEST_SIZE)
X_train=X_train.reset_index()
X_test=X_test.reset_index()
y_train=y_train.reset_index()
y_test=y_test.reset_index()

<div dir="rtl">
<font face="B Nazanin" size=4>
<b>آموزش و ارزیابی مدل</b>
<br>
در این قسمت شما به آماده‌سازی مدل انتخابی و آموزش دسته‌بند می‌پردازید. برای این‌که بتوانید به نتایج بهتری برسید، مقادیر مختلف پارامترها و ابرپارامترهای مدل را با استفاده از داده اعتبارسنجی ارزیابی کنید و بهترین مقادیر را انتخاب کنید. در ارزیابی مدل می‌توانید از معیارهای متفاوتی مانند
      Recall ،Precision ،Accuracy ، Weighted F1-Scoreو F1-Score استفاده کنید.
پس از انتخاب پارامترهای بهینه، می‌توانید مدل نهایی خود را روی تمامی داده‌ها آموزش دهید.
</font>
</div>

<div dir="rtl">
<font face="B Nazanin" size=4>
<b>دسترسی به مدل</b>
<br>
برای ارزیابی مدل شما توسط داده آزمایش (test)، تابعی با نام classify_text بنویسید که به عنوان ورودی دیتافریم تست را دریافت کند و در خروجی یک لیست از اعداد فیلد 'class_id' (از جنس int) متناظر با متن دیتافریم ورودی در ردیف با همان index است را با توجه به موضوع متن تولید کند( test_dataframe.iloc[i] -> results[i] ). این تابع توسط دستیاران آموزشی روی داده‌های آزمایش اجرا شده و نتایج آن برای ارزیابی کیفیت مدل شما استفاده خواهد شد.
</font>
</div>

In [ ]:
def classify_text(test_dataframe):
    labels = ...
    return labels

results = classify_text(test_dataframe)